In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout
from IPython.display import display
from IPython.display import Audio
from functools import partial
import librosa.display
import matplotlib.pyplot as plt
import src.FeatureExtractors as fe
from matplotlib.colors import Normalize
import sklearn
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC


# get our meta data
df = pd.read_csv('datasets/RAVDESS/metadata/RAVDESS.csv',usecols=['actor','actor_sex','emotion','label','filepath'])

# Choose the feature set(s) and add to dataframe

def choose_features(mfcc=40, mel=128):
    scaler = StandardScaler()
    #features = scaler.fit_transform(features)
    # mfcc only
    if mfcc != 'None' and mel == 'None':
        return scaler.fit_transform(np.load(f'datasets/RAVDESS/features/mfcc/mfcc{mfcc}.npy'))
    # mels only
    elif mfcc == 'None' and mel != 'None':
        return scaler.fit_transform(np.load(f'datasets/RAVDESS/features/mel/mel{mel}.npy'))
    # both
    elif mfcc != 'None' and mel != 'None':
        mfcc_frame = np.load(f'datasets/RAVDESS/features/mfcc/mfcc{mfcc}.npy')
        mel_frame = np.load(f'datasets/RAVDESS/features/mel/mel{mel}.npy')
        feature_matrix=np.array([])
        feature_matrix = np.hstack((mfcc_frame, mel_frame))
        return scaler.fit_transform(feature_matrix)



df['features'] = list(choose_features(40,128))





# split data set functions
def random_split(df,test_size=0.2):
    labels = np.array(df['label'])
    features = np.vstack(df['features'])
    
    return train_test_split(features, labels,test_size=test_size,random_state=69)


def split_by_actor_sex(df,train_set='male'):
    train = df
    test = df
    if train_set == 'male':
        train = df[df['actor_sex'] == 'male']
        test = df[df['actor_sex'] == 'female']
    elif train_set == 'female':
        train = df[df['actor_sex'] == 'female']
        test = df[df['actor_sex'] == 'male']

    # , np.vstack(test['features']),np.array(train['label'],np.array(test['label']))
    return np.vstack(train['features']), np.vstack(test['features']), np.array(train['label']),np.array(test['label'])


# CV scoring function -- if I end up wanting it, which maybe I don't?
sklearn.set_config(enable_metadata_routing=True)
def get_model_cv_scores(model, features, labels):
    cv = GroupKFold(5)
    rng = np.random.RandomState(7)
    groups = rng.randint(0, 10, size=len(labels))
    scoring = ['recall_micro', 'precision_micro','f1_micro','balanced_accuracy','roc_auc']
    scores = cross_validate(model, features, labels,scoring=scoring,cv=cv, params={"groups":groups})
    return pd.DataFrame(scores)

# Split the dataset into training and tests -- 
feature_train,feature_test,label_train,label_test = random_split(df, 0.4)

In [2]:
from sklearn.svm import SVC
from sklearn.metrics import recall_score, balanced_accuracy_score
param_grid={
    "C" : [0.1,0.5,0.8,1.0,1.2,1.3],
    "kernel" : ['rbf','poly'],
    "degree" : [2,3,4],
    "gamma" : ["auto","scale"],
    "class_weight" : ["balanced",None]
}


from time import time
cv = GroupKFold(5)
rng = np.random.RandomState(7)
groups = rng.randint(0, 10, size=len(label_train))
grid_search = GridSearchCV(SVC(), param_grid=param_grid,scoring='recall',cv=cv,return_train_score=True)
start = time()
grid_search.fit(feature_train, label_train, groups=groups)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)
use_cols = ['param_C','param_class_weight', 'param_degree', 'param_gamma', 'param_kernel',
        'mean_test_score','mean_train_score','std_test_score', 'rank_test_score']
gr = pd.DataFrame(grid_search.cv_results_)[use_cols]
gr.sort_values(by='rank_test_score',inplace=True)
gr.drop_duplicates(subset=['param_kernel','rank_test_score'],inplace=True)

GridSearchCV took 23.14 seconds for 28 candidate parameter settings.


In [3]:
gr

,param_C,param_class_weight,param_gamma,mean_test_score,mean_train_score,std_test_score,rank_test_score
20,1.5,balanced,auto,0.707572,0.873410,0.058602,1
17,1.2,balanced,scale,0.700429,0.880893,0.071225,2
12,1.0,balanced,auto,0.687095,0.827345,0.070048,3
13,1.0,balanced,scale,0.687095,0.841677,0.070048,3
16,1.2,balanced,auto,0.674595,0.844142,0.074307,5
9,0.8,balanced,scale,0.671257,0.813426,0.072665,6
8,0.8,balanced,auto,0.669704,0.798868,0.052861,7
5,0.5,balanced,scale,0.651614,0.772117,0.086377,8
24,2.0,balanced,auto,0.642496,0.923925,0.196964,9
4,0.5,balanced,auto,0.639114,0.745747,0.093549,10


In [4]:
class TableBuilder:

    def __init__(self):

        pass

    def top_five_table(self, recs):

        cols = ['rank', 'kernel', 'L2Mult','test recall', 'train recall']
        cell_text = []
        for i in range(5):
            cell_text.append([recs['rank'].iloc[i], 
                              recs['kernel'].iloc[i],
                              round(recs['L2Mult'].iloc[i],7),
                              round(recs['test_recall'].iloc[i],2),
                              round(recs['train_recall'].iloc[i],2)])
        return cols, cell_text
    
    def extend_cols(self,rec):
        if rec['kernel']    == 'rbf': return ['gamma','weight']
        elif rec['kernel']  == 'poly':return ['degree','weight']
        elif rec['kernel']  == 'linear':return ['weight']
       
    def extend_cells(self, rec):
        if rec['kernel']   == 'rbf':return [str(rec['gamma']), rec['weight']]
        elif rec['kernel'] == 'poly': return [str(rec['poly_deg']), rec['weight']]
        elif rec['kernel'] == 'linear': return rec['weight']

    
    def format_record_params(self, record):
        cols = ['rank', 'kernel','L2Mult']
        cols.extend(self.extend_cols(record))
        cell_text = [str(record['rank']), record['kernel'],str(round(record['L2Mult'],7))]
        cell_text.extend(self.extend_cells(record))
        return cols, cell_text

    # receive cv inputs as dataframerecord
    def format_current_training_record(self, record, cv_results):
        
        precision = round(cv_results['test_precision_micro'].mean(),2)
        accuracy  = round(cv_results['test_accuracy'].mean(),3)

      
        cell_text=[str(record['rank']), record['kernel'],str(precision),str(accuracy)]
        cols = ['rank', 'kernel', 'precision', 'accuracy']
        # add additional colls/values based on kernel type
        cols.extend(self.extend_cols(record))
        cell_text.extend(self.extend_cells(record))
        return cols, cell_text



In [5]:
from sklearn.metrics import ConfusionMatrixDisplay,RocCurveDisplay,PrecisionRecallDisplay,DetCurveDisplay
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score,precision_score,f1_score,hinge_loss,accuracy_score,d2_absolute_error_score

class TrainingViewer:

    def __init__(self, df, features_train, labels_train,features_test, labels_test):
        
        self.df = self.arrange_columns(df)
        
        self.model = SVC()
        self.current_record_idx = 0

        self.features = features_train
        self.labels = labels_train

        self.features_train = features_train
        self.labels_train = labels_train

        # Testing only
        self.features_test = features_test
        self.labels_test = labels_test
        self.test_model = None
        self.test_mode = 0

        self.predictions = []


    def arrange_columns(self, in_df):
        df = in_df.copy()
        df['rank']=list(range(1,len(df)+1))
        df.drop(columns=['rank_test_score'],inplace=True)
        df.columns=['L2Mult','weight','poly_deg','gamma','kernel','test_recall','train_recall', 'test_stdev','rank']
        df = df[['rank','kernel','L2Mult','weight','poly_deg','gamma','test_recall','train_recall', 'test_stdev']]
        return df
    

    def get_top_five_records(self):
        return self.df.iloc[0:5]


    def print_records_to_table(self):
        tb = TableBuilder()
        cols, cell_text = tb.top_five_table(self.get_top_five_records())

        fig,ax = plt.subplots(figsize=(3,2.5))
        fig.suptitle("Top Five Results")
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        fig.canvas.header_visible = False
        
        table = ax.table(cellText=cell_text,colLabels=cols,loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1.25,2)
        plt.close()
        return fig
    

    def show_box_plot(self):
        by_kernel = {'rbf' : self.df['test_recall'][self.df['kernel'] == 'rbf'],
             #'linear':  self.df['test_recall'][self.df['kernel'] == 'linear'],
             'poly':  self.df['test_recall'][self.df['kernel'] == 'poly']}

        bx=pd.DataFrame(by_kernel,index=None)
        fig, ax = plt.subplots( figsize=(3,3),layout="tight",num=None)
    
        ax.set_title("Kernel Boxplot")
        fig.canvas.header_visible = False

        fig.canvas.toolbar_position = 'bottom'
        ax.grid(True)    
        line = ax.boxplot(bx.describe(),positions=[2,4], widths=1.5, patch_artist=True,
                        showmeans=False, showfliers=False,tick_labels=['rbf','poly'],
                        medianprops={"color": "white", "linewidth": 0.5},
                        boxprops={"facecolor": "C0", "edgecolor": "white",
                                "linewidth": 0.5},
                        whiskerprops={"color": "C0", "linewidth": 1.5},
                        capprops={"color": "C0", "linewidth": 1.5})
        plt.close()
        return fig
  
  
  ####################################################################################
    
    def select_record(self,idx):
        plt.close()
        self.current_record_idx = idx
        self.fit_model_with_record(idx)

    
    def apply_record_to_model(self, rec_index=0):
        if self.df['kernel'].iloc[rec_index] == 'rbf':
            return SVC(C=self.df['L2Mult'].iloc[rec_index],kernel='rbf',gamma=self.df['gamma'].iloc[rec_index],class_weight=self.df['weight'].iloc[rec_index])
        elif self.df['kernel'].iloc[rec_index] == 'poly':
            return SVC(C=self.df['L2Mult'].iloc[rec_index],kernel='poly',degree=self.df['poly_deg'].iloc[rec_index],class_weight=self.df['weight'].iloc[rec_index] )
        elif self.df['kernel'].iloc[rec_index] == 'linear':
            return SVC(C=self.df['L2Mult'].iloc[rec_index],kernel='linear',class_weight=self.df['weight'].iloc[rec_index] )
        else:
            return None


    def fit_model_with_record(self, rec_index=0):    
        self.model = self.apply_record_to_model(rec_index=rec_index)
        self.model = self.model.fit(self.features,self.labels)
        return self.model
       
    def set_testing_model(self):
        self.test_model = self.model
        self.test_mode = 1
        self.predictions = self.test_model.predict(self.features_test)






    def get_model_cv_scores(self):
        scoring = ['recall_micro', 'precision_micro','accuracy']
        scores = cross_validate(self.model, self.features, self.labels,scoring=scoring)
        res = pd.DataFrame(scores)
        return res


    def replace_class_weight(self):
        rec = self.df.iloc[self.current_record_idx].copy()
        if rec["weight"] == None: rec["weight"] = "unbal"
        return rec
    

    def show_curr_record_params(self):
        
        tb = TableBuilder()
        cols, cell_text = tb.format_record_params(self.replace_class_weight())
        fig,axs = plt.subplots(figsize=(2,2))
        fig.suptitle("Current Record Params")
        fig.canvas.header_visible = False
        fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        
        table = axs.table( colLabels=cols,cellText=[cell_text],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(2,2)
        plt.close()
        return fig


    def show_record_stats_table(self):
        tb = TableBuilder()
        cols, cell_text = tb.format_current_training_record(self.replace_class_weight(),
                                                            self.get_model_cv_scores())
         
        fig,axs = plt.subplots(figsize=(3,2))
        fig.suptitle("Selected Record")
        fig.canvas.header_visible = False
        fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        
        table = axs.table( colLabels=cols,cellText=[cell_text],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(2,2)
        plt.close()
        return fig


    def show_confusion_matrix_train(self):
        fig,axs = plt.subplots(figsize=(3,3), layout="constrained")
        axs.set_title("Confusion Matrix Train")
        ConfusionMatrixDisplay.from_estimator(self.model, self.features_train, self.labels_train,ax=axs,colorbar=False)
        plt.close()
        return fig

    def show_confusion_matrix_test(self):
        fig,axs = plt.subplots(figsize=(3,3), layout="constrained")
        axs.set_title("Confusion Matrix Test")
        if self.test_mode == 1:
            ConfusionMatrixDisplay.from_predictions(self.labels_test,self.predictions,ax=axs,colorbar=False,cmap="magma")
        plt.close()
        return fig

    def show_ROC(self):
        
        fig, ax = plt.subplots(figsize=(4,4),layout="constrained")
        fig.canvas.header_visible = False
        ax.set_title("ROC Curve")
        RocCurveDisplay.from_estimator(self.model, self.features_train, self.labels_train,ax=ax,name="ROC Training") 
        
        if self.test_mode == 1:
            pred = self.test_model.decision_function(self.features_test)
            RocCurveDisplay.from_predictions(self.labels_test,pred,ax=ax,name="ROC Test")
        
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.close()
        return fig


    def show_DET(self):
        
        fig, ax = plt.subplots(figsize=(4,3),layout="constrained")
        ax.set_title("DET Curve")
        fig.canvas.header_visible = False
        DetCurveDisplay.from_estimator(self.model, self.features_train, self.labels_train,ax=ax)
        if self.test_mode == 1:
            pred = self.test_model.decision_function(self.features_test)
            DetCurveDisplay.from_predictions(self.labels_test,pred,ax=ax)
        plt.ylabel('False Negative Rate')
        plt.xlabel('False Positive Rate')
        plt.close()
        return fig

    def show_precision_recall(self):

        fig, ax = plt.subplots(figsize=(4,3),layout="constrained")
        fig.canvas.header_visible = False
        ax.set_title("Precision Recall Curve (PRC)")
        PrecisionRecallDisplay.from_estimator(self.model, self.features_train, self.labels_train,ax=ax,name="PRC Train",plot_chance_level=True) 
        
        if self.test_mode == 1:
            pred = self.test_model.decision_function(self.features_test)
            PrecisionRecallDisplay.from_predictions(self.labels_test,pred,ax=ax,name="PRC Test",plot_chance_level=True)
        
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.close()
        return fig

    def get_train_metrics(self):
        # obtain scores
        
        scoring = ['recall','recall_micro','precision_micro','d2_absolute_error_score']
        scores = cross_validate(self.model, self.features, self.labels,scoring=scoring)
        
        cols = ['recall','recall_micro','precision_micro','d2_error']
        cells = []
        cells.append(round(scores['test_recall'].mean(),2))
        cells.append(round(scores['test_recall_micro'].mean(),2))
        
        cells.append(round(scores['test_precision_micro'].mean(),2))
        #cells.append(round(scores['test_f1_micro'].mean(),2))
        cells.append(round(scores['test_d2_absolute_error_score'].mean(),2))

        return cols, cells


    def get_test_metrics(self):
        pred = self.test_model.decision_function(self.features_test)
        cols = ['recall','recall_micro','precision','d2_error']
        cells = []
        cells.append(round(recall_score(self.labels_test, self.predictions),3))
        cells.append(round(recall_score(self.labels_test, self.predictions,average='micro'),3))
       # cells.append(round(accuracy_score(self.labels_test, self.predictions),3))
        cells.append(round(precision_score(self.labels_test, self.predictions,average='micro'),3))
        cells.append(round(d2_absolute_error_score(self.labels_test,self.predictions),3))
        
        return cols, cells
    
    def show_train_metrics(self):
        
        cols, cell_text = self.get_train_metrics()
        rows = ["Train Results"]
        fig,axs = plt.subplots(figsize=(3,2))
        fig.suptitle("Model Performance")
        fig.canvas.header_visible = False
        fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        
        table = axs.table( colLabels=cols,rowLabels=rows,cellText=[cell_text],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(2,2)
        plt.close()
        return fig

    def show_test_metrics(self):
        cols , train_row = self.get_train_metrics()
        _, test_row = self.get_test_metrics()
        
        
        rows = ["Training","Testing"]
        fig,axs = plt.subplots(figsize=(3,2))
        fig.suptitle("Model Performance")
        fig.canvas.header_visible = False
        fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        
        table = axs.table( colLabels=cols,rowLabels=rows,cellText=[train_row, test_row],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(2,2)
        plt.close()
        return fig



# populate the training results records
tr = TrainingViewer(gr,feature_train, label_train,feature_test,label_test)
tr.fit_model_with_record(0)
# prevent plot from displpaying automatically
plt.ioff() 


In [6]:

rec_sel_label = widgets.Label(value="Select Record to View")

rec_sel_dropdown = widgets.Dropdown(
    options=['1','2','3','4','5'],
    value='1',
    description='',
    disabled=False
)


# Handlers
params_out = widgets.Output()
record_out = widgets.Output()
roc_out = widgets.Output()
det_out = widgets.Output()
stats_out =  widgets.Output()
cm_out = widgets.Output()
bxp_out = widgets.Output()
prec_recall_out = widgets.Output()
test_metrics_out = widgets.Output()

run_oos_button  = widgets.Button(description='Test model on out of sample data')

record_box = widgets.VBox([params_out,rec_sel_label,rec_sel_dropdown,record_out,prec_recall_out])

model_box = widgets.VBox([test_metrics_out,prec_recall_out])
stat_box = widgets.VBox([stats_out,bxp_out,cm_out,run_oos_button])
chart_box = widgets.VBox([roc_out, det_out])
left_box = record_box


def initialize():
    
    with params_out:
        params_out.clear_output()
        display(tr.show_curr_record_params())

    with record_out:

        record_out.clear_output()
        display(tr.print_records_to_table())

    with stats_out:
        stats_out.clear_output()
        display(tr.show_train_metrics())

    with cm_out:
        cm_out.clear_output()
        display(tr.show_confusion_matrix_train())

    with roc_out:
        roc_out.clear_output()
        display(tr.show_ROC())

    with det_out:
        det_out.clear_output()
        display(tr.show_DET())

    with bxp_out:
        bxp_out.clear_output()
        display(tr.show_box_plot())

    with prec_recall_out:
        prec_recall_out.clear_output()
        display(tr.show_precision_recall())

    


# observers


No such comm: a17d10221e65417e9aa6ffbb7fd75b15
No such comm: a17d10221e65417e9aa6ffbb7fd75b15
No such comm: a17d10221e65417e9aa6ffbb7fd75b15
No such comm: a17d10221e65417e9aa6ffbb7fd75b15
No such comm: a17d10221e65417e9aa6ffbb7fd75b15
No such comm: a17d10221e65417e9aa6ffbb7fd75b15


No such comm: eabbaa8ba039435fb0cb3bc46fabeb83
No such comm: eabbaa8ba039435fb0cb3bc46fabeb83
No such comm: eabbaa8ba039435fb0cb3bc46fabeb83
No such comm: eabbaa8ba039435fb0cb3bc46fabeb83
No such comm: eabbaa8ba039435fb0cb3bc46fabeb83
No such comm: eabbaa8ba039435fb0cb3bc46fabeb83


No such comm: 1936d6a5a0554a80a908b64ee6b6d340
No such comm: 3ca8139a3b404b148b8b6957c4270a04
No such comm: 3ca8139a3b404b148b8b6957c4270a04
No such comm: 3ca8139a3b404b148b8b6957c4270a04
No such comm: 3ca8139a3b404b148b8b6957c4270a04
No such comm: 3ca8139a3b404b148b8b6957c4270a04
No such comm: 3ca8139a3b404b148b8b6957c4270a04


No such comm: ff2336287fdd475b92d6baa4a80ce432
No such comm: ff2336287fdd475b92d6baa4a80ce432
No such comm: ff2336287fdd475b92d6baa4a80ce432
No such comm: ff2336287fdd475b92d6baa4a80ce432
No such comm: ff2336287fdd475b92d6baa4a80ce432
No such comm: ff2336287fdd475b92d6baa4a80ce432


No such comm: 9e6dabf498814bbab3c8528086989046
No such comm: 9e6dabf498814bbab3c8528086989046
No such comm: 9e6dabf498814bbab3c8528086989046
No such comm: 9e6dabf498814bbab3c8528086989046
No such comm: 9e6dabf498814bbab3c8528086989046
No such comm: 9e6dabf498814bbab3c8528086989046
No such comm: 4ba960e663d941f8b9ae3053be1396bc
No such comm: 4ba960e663d941f8b9ae3053be1396bc
No such comm: 4ba960e663d941f8b9ae3053be1396bc
No such comm: 4ba960e663d941f8b9ae3053be1396bc
No such comm: 4ba960e663d941f8b9ae3053be1396bc
No such comm: 4ba960e663d941f8b9ae3053be1396bc


No such comm: 90569adeedba4bb28a8fd002101cca1b
No such comm: 90569adeedba4bb28a8fd002101cca1b
No such comm: 90569adeedba4bb28a8fd002101cca1b
No such comm: 90569adeedba4bb28a8fd002101cca1b
No such comm: 90569adeedba4bb28a8fd002101cca1b
No such comm: 90569adeedba4bb28a8fd002101cca1b


No such comm: b99f52e794714b07996f7a405889521d
No such comm: b99f52e794714b07996f7a405889521d
No such comm: b99f52e794714b07996f7a405889521d
No such comm: b99f52e794714b07996f7a405889521d
No such comm: b99f52e794714b07996f7a405889521d
No such comm: b99f52e794714b07996f7a405889521d


In [7]:
def select_record_to_view(dfx,names):
    left_box = record_box
    val = int(names.new) - 1
    dfx.select_record(val)
  
    with params_out:
        params_out.clear_output()
        display(tr.show_curr_record_params())
    
    with stats_out:
        stats_out.clear_output()
        display(dfx.show_train_metrics())

    with cm_out:
        cm_out.clear_output()
        display(dfx.show_confusion_matrix_train())

    with roc_out:
        roc_out.clear_output()
        display(dfx.show_ROC())

    with det_out:
        det_out.clear_output()
        display(dfx.show_DET())

    with prec_recall_out:
        prec_recall_out.clear_output()
        display(dfx.show_precision_recall())

    plt.close('all')
    

def test_model_on_oos(dfx,val):
    
    dfx.set_testing_model()
    with params_out:
        params_out.clear_output()
        display(tr.show_curr_record_params())

    with cm_out:
        cm_out.clear_output()
        display(dfx.show_confusion_matrix_train())

    with roc_out:
        roc_out.clear_output()
        display(dfx.show_ROC())

    with det_out:
        det_out.clear_output()
        display(dfx.show_DET())
    
    with prec_recall_out:
        prec_recall_out.clear_output()
        display(dfx.show_precision_recall())
    
    with bxp_out:
        bxp_out.clear_output()
        display(dfx.show_confusion_matrix_test())

    with record_out:
        record_out.clear_output() 
        display(dfx.show_test_metrics())
        

    with test_metrics_out:
        test_metrics_out.clear_output()
        


    plt.close('all')     


initialize()



rec_sel_dropdown.observe(partial(select_record_to_view,tr),names='value')
run_oos_button.on_click(partial(test_model_on_oos, tr))

gridsearch_panel = widgets.HBox([left_box, stat_box,chart_box])

gridsearch_panel


In [8]:
plt.close()

In [9]:
#tr.test_mode = 0